In [ ]:
print('starting script')
from galsbi import GalSBI
import matplotlib.pyplot as plt
import healpy as hp
import numpy as np
import time
print("GalsBI version:")
maskpath = '/Users/voehl/Teaching/students/noah/circular_1000sqd_nside256.fits'
mask = hp.fitsfunc.read_map(maskpath)
patch_pixels = np.where(mask>0)[0]
print('patch_pixels', patch_pixels)
n_pixels = int(np.sum(mask[mask>0]))
npix = len(mask)
submasks = []
catalog_names = []
pixels_per_mask = 10
cats = []
def process_healpix_submask(submask, catalog_name):
    model = GalSBI("Fischbacher+24")
    model(healpix_map=submask, catalog_name=catalog_name,gals_mag_max = 25)
    cats.append(model.load_catalogs(combine=True))

for i in range(n_pixels // pixels_per_mask + 1):
    print(f"Creating submask {i} out of {n_pixels // pixels_per_mask + 1}")
    submask = np.zeros(npix)
    pixels = patch_pixels[i*pixels_per_mask : (i+1)*pixels_per_mask]
    submask[pixels] = 1
    submasks.append(submask)
    catalog_names.append(f"catalogs_pixelbatch{i}")

for i in range(len(submasks)):
    print(f"Processing submask {i} out of {len(submasks)}")
    tic = time.time()
    process_healpix_submask(submasks[i], catalog_name=catalog_names[i])
    toc = time.time()
    print(f"Time elapsed: {toc-tic} s")


In [ ]:
print(cats[0]['ucat galaxies'].dtype, cats[1]['ucat galaxies'].dtype)
galaxy_list = [cats[i]['ucat galaxies'] for i in range(len(cats))]
galaxies = np.concatenate(galaxy_list)

#galaxies = cats['ucat galaxies']
z_data = galaxies['z']
kappa = galaxies['kappa']
print(kappa)

plt.figure(figsize=(3,2))
plt.hist(galaxies['z'], bins=100)
plt.xlabel("$z$")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from scipy.stats import norm

# Generate synthetic data
np.random.seed(42)
data = z_data.reshape(-1, 1)
numbins = 5
# Define initial means: four equally spaced points across the data range
initial_means = np.array([0.2,0.4,0.6,1.0,2.2]).reshape(-1, 1)

# Fit GMM with 4 components and manually initialized means
gmm = GaussianMixture(n_components=numbins, means_init=initial_means, random_state=42)
gmm.fit(data)

# Predict the component labels
# Get probability that each sample belongs to each Gaussian
probabilities = gmm.predict_proba(data)  # Shape: (N, 4)
print(probabilities)

# Soft assignment: Randomly assign based on probabilities
labels = np.array([np.random.choice(numbins, p=prob) for prob in probabilities])

# Split data into numbins groups
subdistributions = [data[labels == i] for i in range(numbins)]
for distr in subdistributions:
    print(len(distr))

# Plot the results
plt.hist(data, bins=100, alpha=0.5, label="Original Data", color="gray", density=False)
x = np.linspace(min(data), max(data), 1000).reshape(-1, 1)
densities = np.exp(gmm.score_samples(x))
plt.plot(x, densities, label="GMM Fit", color="black")

# Plot individual components
for i in range(numbins):
    plt.hist(subdistributions[i], bins=50, alpha=0.6, label=f"Component {i+1}", density=False)

plt.scatter(gmm.means_, [0] * numbins, color='red', marker='x', s=100, label="Final Means")
plt.legend()
plt.show()

plt.figure()
plt.hist(data, bins=50, alpha=0.5, label="Original Data", color="gray", density=True)
x = np.linspace(min(data), max(data), 1000).reshape(-1, 1)
densities = np.exp(gmm.score_samples(x))
plt.plot(x, densities, label="GMM Fit", color="black")
for i in range(numbins):
    plt.plot(x,norm.pdf(x, gmm.means_[i], np.sqrt(gmm.covariances_[i])).ravel(), label=f"Component {i+1}")
plt.show()


In [ ]:
redshift_bin = 4
selection = galaxies[labels == redshift_bin]
np.save("catalogue_100sqd", selection)
print(selection)
import healpy as hp
print(hp.nside2resol(64, arcmin=True))

In [ ]:
selection = np.load("catalogue_100sqd.npy", allow_pickle=True)

In [ ]:
print(selection['ra'],selection['dec'])


plt.figure(figsize=(3,2))
plt.hist(selection['z'], bins=100)
plt.xlabel("$z$")
plt.show()

In [ ]:

pixscale = 0.263
sizes_in_arcsec = selection['r50'] * pixscale   # arcsec
print(sizes_in_arcsec)
lon_center = np.mean(selection['ra'])
lat_center = np.mean(selection['dec'])
lon_range = 1
lat_range = 1
ra = selection['ra']
dec = selection['dec']

theta = np.radians(90.0 - dec)  # theta = 90 - dec
phi = np.radians(ra)            # phi = ra

# Define the nside parameter (must be a power of 2)
nside = 128

# Initialize a HEALPix map with zeros
size_map = np.zeros(hp.nside2npix(nside))
numbers = np.zeros(hp.nside2npix(nside))
# Convert RA and Dec to pixel indices
pixels = hp.ang2pix(nside, theta, phi)

# Fill the HEALPix map with galaxy sizes
for pix, size in zip(pixels, sizes_in_arcsec):
    size_map[pix] += size
    numbers[pix] += 1

mean_size_map = size_map / numbers
overall_mean, overall_std = np.mean(sizes_in_arcsec), np.std(sizes_in_arcsec)
print(overall_mean, overall_std)
hp.mollview(mean_size_map, title="Mean Galaxy Size", unit="arcsec", cbar=True)

hp.mollview(numbers, title="Number of Galaxies", unit="galaxies", cbar=True)

hp.cartview(lonra=[lon_center-lon_range / 2, lon_center+lon_range/2], latra=[lat_center-lat_range/2, lat_center+lat_range/2], title="Galaxy Distribution", cbar=False)
hp.projscatter(selection['ra'],selection['dec'], s=10*sizes_in_arcsec,lonlat=True)
ax = plt.gca()
ax.set_xlabel('Right Ascension (degrees)')
ax.set_ylabel('Declination (degrees)')
plt.show()

In [ ]:
import pyccl as ccl
omega_m = 0.31
omega_b = 0.046
omega_c = omega_m - omega_b
cosmo = ccl.Cosmology(Omega_c = omega_c, Omega_b = omega_b, h = 0.7, sigma8 = 0.8, n_s = 0.97)
z, nz = x.ravel(), norm.pdf(x, gmm.means_[redshift_bin], np.sqrt(gmm.covariances_[redshift_bin])).ravel()

lens = ccl.WeakLensingTracer(cosmo, dndz=(z, nz))

ell = np.arange(2, 10000)
cl_ee = ccl.angular_cl(cosmo, lens, lens, ell)
print(gmm.means_[redshift_bin])
np.savetxt('cl_kappa_mean_{:.2f}.txt'.format(gmm.means_[redshift_bin][0]), np.column_stack((ell, cl_ee)))


In [ ]:
cl_ee = np.concatenate((np.zeros(2), cl_ee))
kappamap = hp.synfast(cl_ee, 128, new=True) # also make lognormal map
hp.mollview(kappamap, title="Convergence Map", cbar=True)
hp.cartview(kappamap,lonra=[lon_center-10-lon_range/2, lon_center-10+lon_range/2], latra=[lat_center-20-lat_range/2, lat_center-20+lat_range/2], title="kappa", cbar=True)


In [ ]:
galaxy_pixels = hp.ang2pix(256, selection['ra'], selection['dec'], lonlat=True)
selection['kappa'] = kappamap[galaxy_pixels]

In [ ]:
lensed_sizes = sizes_in_arcsec * (1 + selection['kappa'])
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(20,10))
hp.cartview(lonra=[lon_center-lon_range / 2, lon_center+lon_range/2], latra=[lat_center-lat_range/2, lat_center+lat_range/2], title="Galaxy Distribution", cbar=False, fig=fig, sub=(1,2,1))
hp.projscatter(selection['ra'],selection['dec'], s=10*sizes_in_arcsec,lonlat=True)
hp.cartview(lonra=[lon_center-lon_range / 2, lon_center+lon_range/2], latra=[lat_center-lat_range/2, lat_center+lat_range/2], title="Lensed Galaxy Distribution", cbar=False, fig=fig, sub=(1,2,2))
hp.projscatter(selection['ra'],selection['dec'], s=10*lensed_sizes,lonlat=True)
print(np.mean(sizes_in_arcsec), np.mean(lensed_sizes))

plt.figure()
plt.hist(sizes_in_arcsec, bins=100, alpha=0.5, label="Original Sizes", color="gray", density=False)
plt.hist(lensed_sizes, bins=100, alpha=0.5, label="Lensed Sizes", color="blue", density=False)
plt.xlim(0, 2)
plt.legend()
plt.show()



compare mean size in each pixel, i.e. get all galaxies in one pixel and look at their mean, median, weighted mean size - how does it vary? compare to kappa assigned to this pixel. Increase pixel size of kappa map. 
can we weigh by overall size distribution? what would this mean? weigh most common galaxy size more or less?
null hypothesis: mean is not changed